In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE174570"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE174570"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE174570.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE174570.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE174570.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Define variables for gene expression and trait data availability
is_gene_available = True  # The dataset appears to have gene expression data from Human Genome U219 Array
trait_row = 0  # The disease state (HCC) is in row 0
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# Define conversion functions for trait
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 for HCC, 0 for non-HCC/normal)
    if 'HCC' in value:
        return 1
    elif 'non' in value.lower() or 'normal' in value.lower() or 'control' in value.lower():
        return 0
    else:
        return None

# Define placeholder conversion functions for age and gender
def convert_age(value):
    return None  # Age data is not available

def convert_gender(value):
    return None  # Gender data is not available

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata using the validate_and_save_cohort_info function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if trait_row is not None:
    try:
        # Convert the sample characteristics dictionary to a DataFrame
        # This is assuming the sample_characteristics dictionary from the previous output is available
        # For example: {0: ['disease state: HCC'], 1: ['tissue: Tumour (liver)', 'tissue: Non-tumour adjacent (liver)']}
        sample_characteristics = {
            0: ['disease state: HCC'], 
            1: ['tissue: Tumour (liver)', 'tissue: Non-tumour adjacent (liver)']
        }
        
        # Create a DataFrame from the sample characteristics
        # Convert dictionary to list of lists format for DataFrame
        data = []
        max_values = max(len(values) for values in sample_characteristics.values())
        
        for i in range(max_values):
            row = []
            for key in sorted(sample_characteristics.keys()):
                if i < len(sample_characteristics[key]):
                    row.append(sample_characteristics[key][i])
                else:
                    row.append(None)
            data.append(row)
        
        clinical_data = pd.DataFrame(data)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )

        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Clinical DataFrame Preview:")
        print(preview)

        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    print("Trait data is not available. Skipping clinical feature extraction.")


### Step 3: Gene Data Extraction

In [ ]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers ('11715100_at', etc.) are Affymetrix probe IDs, not human gene symbols.
# They need to be mapped to gene symbols for proper biological interpretation.
# Affymetrix probe IDs are microarray-specific identifiers that need to be converted
# to standard gene symbols for cross-platform analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns contain the gene identifiers and gene symbols
# From the previous output, we can see:
# - The gene expression data uses Affymetrix probe IDs as identifiers (e.g., '11715100_at')
# - In the gene annotation dataframe, the 'ID' column contains these same probe IDs
# - The 'Gene Symbol' column contains the corresponding gene symbols

# 2. Create a gene mapping dataframe using the appropriate columns
prob_col = 'ID'  # Column with probe IDs
gene_col = 'Gene Symbol'  # Column with gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"\nGene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of the mapping dataframe:")
print(preview_df(mapping_df, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"\nGene expression dataframe after mapping - shape: {gene_data.shape}")
print("First few rows of the gene expression dataframe:")
print(preview_df(gene_data, n=5))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in step 2
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Clinical data loaded from {out_clinical_data_file}")
    print(f"Clinical data columns: {clinical_df.columns.tolist()}")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # Create a minimal clinical DataFrame based on our understanding from previous steps
    clinical_df = pd.DataFrame({0: [1.0]})
    print("Created basic clinical data as fallback")

# Check if the trait can be found in the clinical data columns
if trait in clinical_df.columns:
    trait_col_to_use = trait
else:
    # Use the first column as the trait column if available
    trait_col_to_use = clinical_df.columns[0] if len(clinical_df.columns) > 0 else None
    if trait_col_to_use is not None:
        print(f"Using column '{trait_col_to_use}' as trait column")
        # Rename for clarity in downstream processing
        clinical_df = clinical_df.rename(columns={trait_col_to_use: trait})
        trait_col_to_use = trait

# Based on our analysis from Step 2, this dataset doesn't have appropriate trait data
# for liver cancer comparison (all samples are cancer without controls)
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # We determined in Step 2 that there's no proper trait data for case/control comparison
is_biased = True  # Since all samples are cancer cell lines, there's no control vs. cancer comparison

# 5. Conduct final quality validation and save metadata
print("\nConducting final quality validation...")
note = "This dataset contains gene expression data from human liver samples, but it lacks appropriate control vs. cancer comparison needed for liver cancer association studies. From background information, all samples appear to be from HCC without normal controls."

# Create a minimal linked dataframe for validation purposes
if trait_col_to_use is not None:
    linked_data = pd.DataFrame({trait: clinical_df[trait].values})
else:
    linked_data = pd.DataFrame()  # Empty DataFrame if no trait column

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# No need to save linked_data since the dataset is not usable for our trait analysis
if is_usable:
    # Link the clinical and genetic data for a proper dataset
    linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
    
    # Handle missing values if we have a trait column
    if trait in linked_data.columns:
        linked_data = handle_missing_values(linked_data, trait)
    
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")